<a href="https://colab.research.google.com/github/rgriveros/ENTREGA-FINAL_DS2_RRiveros/blob/main/EntregaFinal_DS2_RRiveros.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **EntregaFinal — Proyectos Mineros en Argentina**
## Autor: **Gabriel Riveros Lobos**  
## Fecha: 13/11/2025  
## Objetivo del notebook: **Predecir si un proyecto minero alcanza etapa avanzada (clasificación binaria).**  
## Métrica principal: AUC (Area Under ROC).  

## **Contexto:**
Cartera de proyectos mineros con atributos técnicos, geográficos y de propiedad. Los equipos de evaluación y toma de decisiones necesitan priorizar recursos y planificar inversiones.

## **Objetivo:**
Construir un modelo de clasificación para predecir si un proyecto alcanzará una etapa avanzada (sí/no), facilitando priorización de inversión y asignación de recursos operativos.
Pregunta de negocio: ¿Qué proyectos de la cartera tienen alta probabilidad de alcanzar una etapa avanzada y, por tanto, merecen priorización de inversión?

## **Audiencia beneficiada:**
Gerencias de proyectos, analistas de cartera e inversores que requieren señales tempranas sobre proyectos con mayor probabilidad de avance.

## **Métrica principal:**
AUC ROC — elegida por su robustez ante desequilibrios en clases y porque mide la capacidad global del modelo para distinguir proyectos que avanzan de los que no.

### Justificación corta de la métrica
AUC ROC está indicada porque: Maneja bien clases desbalanceadas sin requerir un umbral fijo. Permite comparar modelos independientemente del coste asociado a falsos positivos/negativos; luego se puede fijar umbral según trade-off operativo.

## **Resumen ejecutivo:**
Este notebook presenta la carga y limpieza de datos, la ingeniería de features, la comparación de modelos (baseline vs tree-based), la optimización ligera de hiperparámetros y la explicación del modelo final con SHAP. El entregable incluye el modelo final serializado, métricas comparativas y recomendaciones operativas para priorizar la cartera.


---
## **Instrucciones rápidas:**  
Clonar el repo.
pip install -r requirements.txt..
Ejecutar la celda "2 — Preparación" del notebook (descarga automática vía CKAN y guarda data/dataset.csv).


# 2. Requisitos, entorno y reproducibilidad

Objetivo: dejar configurado el entorno mínimo necesario para que el notebook sea reproducible y ejecutable en Colab y en cualquier máquina local con Python. Esto incluye:
- Declarar y fijar la semilla global (RANDOM_STATE).
- Importar librerías clave y mostrar versiones para trazabilidad.
- Definir y crear la estructura de carpetas usada por el proyecto.
- Generar archivos iniciales: `README_colab.txt` y `data/sample_input.csv` (muestra vacía), listos para subir al repo.



In [1]:
# 2.2 Imports, seed, paths y creación de estructura
import sys, os, platform
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib, matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import joblib
from datetime import datetime

# Semilla reproducible
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Imprimir versiones clave para reproducibilidad
print("Python:", sys.version.splitlines()[0])
print("Platform:", platform.platform())
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Matplotlib:", matplotlib.__version__)
print("Seaborn:", sns.__version__)
print("Scikit-learn:", sklearn.__version__)
print("Joblib:", joblib.__version__)
print("RANDOM_STATE set to", RANDOM_STATE)

Python: 3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]
Platform: Linux-6.6.105+-x86_64-with-glibc2.35
NumPy: 2.0.2
Pandas: 2.2.2
Matplotlib: 3.10.0
Seaborn: 0.13.2
Scikit-learn: 1.6.1
Joblib: 1.5.2
RANDOM_STATE set to 42


# Carga de Datos y chequeo rápido EDA

Objetivo:

Esta celda descarga el dataset público desde la API CKAN de datos.gob.ar (package_search → package_show → recurso CSV/XLSX), guarda el archivo en la carpeta local data/ del repositorio y genera un snapshot local para trazabilidad. No requiere acceso a Google Drive. Variables editables: QUERY, ROWS, VERIFY_SSL. Mantener VERIFY_SSL = True; usar False solo si el entorno falla por certificado (documentar el uso). Dependencias: requests, pandas, openpyxl (si hay Excel).


In [7]:
# 2. Descarga por API CKAN -> guarda en ./data/ -> lee y snapshot
import requests
from pathlib import Path
import pandas as pd
import time

# ---------- EDITAR SOLO ESTAS VARIABLES ----------
CKAN_BASE = "https://datos.gob.ar/api/3/action"
QUERY = "Proyectos mineros metalíferos y de litio"
ROWS = 5
VERIFY_SSL = False   # True recomendado; False solo si hay error certificado y lo documentás
# -------------------------------------------------

DATA_DIR = Path("data")
DATA_DIR.mkdir(parents=True, exist_ok=True)

def ck_search_and_download(query, rows=5, verify_ssl=True, retries=2, backoff=2):
    for attempt in range(retries + 1):
        try:
            r = requests.get(f"{CKAN_BASE}/package_search", params={"q": query, "rows": rows}, timeout=30, verify=verify_ssl)
            r.raise_for_status()
            results = r.json().get("result", {}).get("results", [])
            if not results:
                raise FileNotFoundError(f"No se encontraron paquetes para la query: '{query}'")
            first = results[0]
            pkg_id = first.get("id") or first.get("name")
            r2 = requests.get(f"{CKAN_BASE}/package_show", params={"id": pkg_id}, timeout=30, verify=verify_ssl)
            r2.raise_for_status()
            pkg = r2.json().get("result", {})
            resources = pkg.get("resources", []) or []
            for res in resources:
                url = (res.get("url") or "").strip()
                if url.lower().endswith((".csv", ".xlsx", ".xls")):
                    return pkg, url
            raise FileNotFoundError("No se encontró recurso CSV/XLSX en el paquete seleccionado.")
        except (requests.RequestException, ValueError) as e:
            if attempt < retries:
                time.sleep(backoff * (attempt + 1))
                continue
            raise RuntimeError(f"Fallo al consultar CKAN: {e}")

pkg_meta, res_url = ck_search_and_download(QUERY, ROWS, VERIFY_SSL)

dst = DATA_DIR / ("dataset_minero.csv" if res_url.lower().endswith(".csv") else "dataset_minero.xlsx")
with requests.get(res_url, stream=True, timeout=120, verify=VERIFY_SSL) as resp:
    resp.raise_for_status()
    with open(dst, "wb") as f:
        for chunk in resp.iter_content(8192):
            if chunk:
                f.write(chunk)

if dst.suffix.lower() == ".csv":
    df = pd.read_csv(dst, low_memory=False, encoding="utf-8")
else:
    df = pd.read_excel(dst, engine="openpyxl")

# Guardar snapshot para trazabilidad
snapshot = DATA_DIR / "data_snapshot_head.csv"
df.head(200).to_csv(snapshot, index=False)

# Salida mínima y reproducible
print("Package:", pkg_meta.get("title") or pkg_meta.get("name"))
print("Fuente (URL):", res_url)
print("Guardado en:", dst)
print("Filas:", df.shape[0], "| Columnas:", df.shape[1])
print(df.isna().sum().sort_values(ascending=False).head(20).to_string())


/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datos.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'datos.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.mecon.gob.ar'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/urllib3/connectionpool.py:1097: 

Package: Cartera de Proyectos Mineros en Argentina del SIACAM
Fuente (URL): https://www.mecon.gob.ar/dataset/Cartera-de-Proyectos-Mineros-Metaliferos-y-Litio-del-SIACAM.xlsx
Guardado en: data/dataset_minero.xlsx
Filas: 325 | Columnas: 17
Unnamed: 16          324
PORCENTAJE (3°)      235
ORIGEN (2°)          233
ORIGEN (3°)          232
CONTROLANTE (3°)     232
CONTROLANTE (2°)     231
PORCENTAJE (2°)      229
PORCENTAJE (1°)      122
ORIGEN (1°)          121
CONTROLANTE (1°)      97
N°                     0
PROVINCIA              0
ESTADO                 0
LATITUD                0
NOMBRE                 0
LONGITUD               0
MINERAL PRINCIPAL      0


### Guardando metadatos del paquete (id, title, url, fecha de descarga) para trazabilidad reproducible

In [9]:
# Guardar metadatos del paquete
import json
from datetime import datetime, timezone
from pathlib import Path

META_PATH = Path("data")
META_PATH.mkdir(parents=True, exist_ok=True)
OUT = META_PATH / "dataset_metadata.json"

def safe_str(x):
    try:
        return str(x)
    except Exception:
        return None

meta = {
    "package_id": safe_str(pkg_meta.get("id") if isinstance(pkg_meta, dict) else pkg_meta),
    "package_title": safe_str(pkg_meta.get("title") if isinstance(pkg_meta, dict) else pkg_meta),
    "resource_url": safe_str(res_url),
    "downloaded_at": datetime.now(timezone.utc).isoformat()
}

try:
    with open(OUT, "w", encoding="utf-8") as f:
        json.dump(meta, f, ensure_ascii=False, indent=2)
    print("Metadatos guardados en:", OUT)
except Exception as e:
    print("Error al guardar metadatos:", type(e).__name__, "-", e)
    print("Contenido meta (fallback):", {k: v for k, v in meta.items()})


Metadatos guardados en: data/dataset_metadata.json


In [10]:
print(df.dtypes)
print(df.head(5))


N°                     int64
NOMBRE                object
LATITUD              float64
LONGITUD             float64
MINERAL PRINCIPAL     object
PROVINCIA             object
ESTADO                object
CONTROLANTE (1°)      object
PORCENTAJE (1°)      float64
ORIGEN (1°)           object
CONTROLANTE (2°)      object
ORIGEN (2°)           object
PORCENTAJE (2°)       object
CONTROLANTE (3°)      object
PORCENTAJE (3°)       object
ORIGEN (3°)           object
Unnamed: 16           object
dtype: object
   N°             NOMBRE  LATITUD  LONGITUD MINERAL PRINCIPAL  PROVINCIA  \
0   1  20 de septiembre   -24.896   -68.136            Hierro      Salta   
1   2           Acazoque  -24.291   -66.378             Plomo      Salta   
2   3   Acoite/Hornillos  -22.305   -65.107             Plomo      Salta   
3   4              Adamo  -41.162   -68.505               Oro  Río Negro   
4   5      Aguas Amargas  -24.685   -66.903             Cobre      Salta   

                ESTADO        CONTRO

## Observación rápida sobre la salida
La lectura funcionó, pero hay columnas que deberían ser numéricas (p. ej. las columnas PORCENTAJE (...)) y aparecen como object — eso suele pasar por valores mezclados (cadenas, comas decimales, símbolos % o celdas vacías/extrañas).

Hay una columna Unnamed: 16 vacía en su mayoría — probable columna residual del Excel que conviene eliminar.

Latitud/Longitud ya son float64 (bien).

Antes de cualquier análisis sería conveniente:
1) inspeccionar las celdas “problemáticas”,
2) normalizar formatos numéricos,
3) convertir a tipos correctos
4) registrar los cambios (snapshot + metadata).

# 4. Ingeniería de features y preprocesado reproducible

In [11]:
# Sección 4: Limpieza no destructiva -> guarda clean CSV, snapshot y metadata ampliada
import re, json, time
from pathlib import Path
from datetime import datetime, timezone
import pandas as pd

# ----- Ajustar si fuera necesario -----
DATA_DIR = Path("data")
RAW_FN = DATA_DIR / "dataset_minero.xlsx"   # cambiar si el raw tiene otro nombre
OUT_CSV = DATA_DIR / "dataset_minero_clean.csv"
SNAPSHOT_CSV = DATA_DIR / "data_snapshot_head_clean.csv"
META_F = DATA_DIR / "dataset_metadata.json"
# -------------------------------------

DATA_DIR.mkdir(parents=True, exist_ok=True)

# 0) cargar (si df ya existe en memoria, lo reutiliza)
if 'df' not in globals():
    if RAW_FN.exists():
        if RAW_FN.suffix.lower() == ".csv":
            df = pd.read_csv(RAW_FN, low_memory=False, encoding="utf-8")
        else:
            df = pd.read_excel(RAW_FN, engine="openpyxl")
    else:
        raise FileNotFoundError(f"Archivo raw no encontrado: {RAW_FN}")

In [12]:


# 1) detectar columnas objetivo
pct_cols = [c for c in df.columns if "PORCENTAJE" in str(c).upper()]
obj_cols = [c for c in df.columns if df[c].dtype == "object" and c not in pct_cols]

print("Columnas detectadas como PORCENTAJE:", pct_cols)
print("Ejemplo de columnas object detectadas:", obj_cols[:8])

# 2) limpieza helper (devuelve float o None)
def clean_numeric_value(x):
    if pd.isna(x):
        return None
    s = str(x).strip()
    if s == "" or s in ("-", "--"):
        return None
    s = s.replace("\xa0", "").replace(" ", "")
    # quitar texto no numérico salvo signos, decimales, exp y %
    s = re.sub(r"[^\d\-,.\+eE%]", "", s)
    if s == "" or s in ("-", "--"):
        return None
    is_pct = s.endswith("%")
    if is_pct:
        s = s[:-1]
    # manejar '.' y ',' coexistentes
    if "," in s and "." in s:
        if s.rfind(",") > s.rfind("."):
            s = s.replace(".", "").replace(",", ".")
        else:
            s = s.replace(",", "")
    else:
        if "," in s and "." not in s:
            s = s.replace(",", ".")
    try:
        v = float(s)
        # si era %, normalizar a 0-1; si nombre de columna indica porcentaje y v>1 asumimos 0-100->0-1
        return v/100.0 if is_pct else v
    except Exception:
        return None

Columnas detectadas como PORCENTAJE: ['PORCENTAJE (1°)', 'PORCENTAJE (2°)', 'PORCENTAJE (3°)']
Ejemplo de columnas object detectadas: ['NOMBRE', 'MINERAL PRINCIPAL', 'PROVINCIA', 'ESTADO', 'CONTROLANTE (1°)', 'ORIGEN (1°)', 'CONTROLANTE (2°)', 'ORIGEN (2°)']


In [13]:
# 3) aplicar a columnas PORCENTAJE y crear columnas *_clean (no sobreescribir)
conversion_report = {}
for c in pct_cols:
    before = int(df[c].notna().sum())
    cleaned = df[c].map(clean_numeric_value)
    after = int(cleaned.notna().sum())
    df[c + "_clean"] = cleaned
    conversion_report[c] = {"before_nonnull": before, "after_numeric": after}

# 4) opcional: aplicar a otras columnas object que el usuario confirme (aquí no se aplica por defecto)
# Si querés convertir columnas adicionales, agrégalas a cols_to_convert
cols_to_convert = []  # p. ej. ["PORCENTAJE (2°)"] si no fue detectada por nombre
for c in cols_to_convert:
    if c in df.columns:
        before = int(df[c].notna().sum())
        cleaned = df[c].map(clean_numeric_value)
        after = int(cleaned.notna().sum())
        df[c + "_clean"] = cleaned
        conversion_report[c] = {"before_nonnull": before, "after_numeric": after}

# 5) eliminar columnas vacías tipo Unnamed si están totalmente vacías
unnamed = [c for c in df.columns if str(c).lower().startswith("unnamed") and df[c].dropna().eq("").all()]
# además columnas con todos NaN
allnan = [c for c in df.columns if df[c].isna().all()]
drop_candidates = sorted(set(unnamed + allnan))
if drop_candidates:
    print("Columnas vacías a dropear (no destructivo todavía):", drop_candidates)
    df = df.drop(columns=drop_candidates, errors='ignore')

In [14]:
# 6) guardar clean CSV y snapshot head
df.to_csv(OUT_CSV, index=False, encoding="utf-8")
df.head(200).to_csv(SNAPSHOT_CSV, index=False, encoding="utf-8")

# 7) metadata ampliada
meta = {
    "package_title": pkg_meta.get("title") if 'pkg_meta' in globals() else None,
    "package_id": pkg_meta.get("id") if 'pkg_meta' in globals() else None,
    "resource_url": res_url if 'res_url' in globals() else None,
    "raw_file": str(RAW_FN),
    "clean_file": str(OUT_CSV),
    "snapshot_file": str(SNAPSHOT_CSV),
    "generated_at": datetime.now(timezone.utc).isoformat().replace("+00:00","Z"),
    "conversion_report": conversion_report,
    "dropped_columns": drop_candidates
}
with open(META_F, "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("Limpieza aplicada (no destructiva). Archivos creados:")
print(" - Clean dataset:", OUT_CSV)
print(" - Snapshot head:", SNAPSHOT_CSV)
print(" - Metadata:", META_F)
print("\nResumen conversion_report (porcentaje columnas):")
print(json.dumps(conversion_report, indent=2, ensure_ascii=False))


Limpieza aplicada (no destructiva). Archivos creados:
 - Clean dataset: data/dataset_minero_clean.csv
 - Snapshot head: data/data_snapshot_head_clean.csv
 - Metadata: data/dataset_metadata.json

Resumen conversion_report (porcentaje columnas):
{
  "PORCENTAJE (1°)": {
    "before_nonnull": 203,
    "after_numeric": 203
  },
  "PORCENTAJE (2°)": {
    "before_nonnull": 96,
    "after_numeric": 30
  },
  "PORCENTAJE (3°)": {
    "before_nonnull": 90,
    "after_numeric": 4
  }
}


## ¿Qué haremos y por qué?

Crear un respaldo del dataset actual sin modificar (backup CSV) para preservar la fuente antes de cualquier cambio.

Reemplazar cada columna original por su columna correspondiente con sufijo _clean (si existe) para aplicar la limpieza validada en la sección anterior. Mantendremos también las columnas _clean en el archivo final para trazabilidad.

Guardar el dataset resultante como data/dataset_minero_clean_applied.csv y actualizar dataset_metadata.json con el registro de cambios (qué columnas se reemplazaron, conteos antes/después, timestamp y ruta del backup).

### Beneficios y trazabilidad

Reversibilidad: el respaldo permite restaurar el estado previo si detectás problemas más adelante.

Auditoría: metadata contiene package id/title, resource_url, columnas reemplazadas y contadores, lo que facilita reproducir y auditar exactamente lo que se cambió.

Seguridad para el revisor: la limpieza ya fue aplicada solo después de la inspección en Sección 3; aquí se realiza la operación irreversible controlada y documentada.

In [15]:
# Sección 4.1 Renombrar *_clean -> sobrescribir columnas originales con backup
import json
from pathlib import Path
from datetime import datetime, timezone
import pandas as pd
import shutil

DATA_DIR = Path("data")
RAW_FN = DATA_DIR / "dataset_minero.xlsx"
BACKUP_RAW_CSV = DATA_DIR / "dataset_minero_raw_backup.csv"
OUT_APPLIED = DATA_DIR / "dataset_minero_clean_applied.csv"
META_F = DATA_DIR / "dataset_metadata.json"

# 4.1.0) cargar df (si no existe en memoria lo leemos del clean intermedio o del raw)
if 'df' not in globals():
    # preferimos el clean intermedio si existe
    candidate_clean = DATA_DIR / "dataset_minero_clean.csv"
    if candidate_clean.exists():
        df = pd.read_csv(candidate_clean, low_memory=False, encoding="utf-8")
    elif RAW_FN.exists():
        if RAW_FN.suffix.lower() == ".csv":
            df = pd.read_csv(RAW_FN, low_memory=False, encoding="utf-8")
        else:
            df = pd.read_excel(RAW_FN, engine="openpyxl")
    else:
        raise FileNotFoundError("No se encontró ningún dataset intermedio ni raw para aplicar los cambios.")

In [16]:
# 4.1.1) identificar columnas *_clean y sus originales
clean_cols = [c for c in df.columns if isinstance(c, str) and c.endswith("_clean")]
mapping = {}
for c in clean_cols:
    orig = c[:-6]  # quitar sufijo "_clean"
    if orig in df.columns:
        mapping[orig] = c

if not mapping:
    raise RuntimeError("No se encontraron columnas *_clean que correspondan a columnas originales. Nada que aplicar.")

print("Columnas a reemplazar (original -> *_clean):")
for orig, clean in mapping.items():
    print(f" - {orig}  <-  {clean}")

# 4.1.2) crear backup CSV del estado actual (antes de aplicar)
df.to_csv(BACKUP_RAW_CSV, index=False, encoding="utf-8")
print("Backup creado en:", BACKUP_RAW_CSV)

Columnas a reemplazar (original -> *_clean):
 - PORCENTAJE (1°)  <-  PORCENTAJE (1°)_clean
 - PORCENTAJE (2°)  <-  PORCENTAJE (2°)_clean
 - PORCENTAJE (3°)  <-  PORCENTAJE (3°)_clean
Backup creado en: data/dataset_minero_raw_backup.csv


In [17]:
# 4.1.3) aplicar reemplazo: para cada mapping, mover valores de columna_clean a columna original
replaced = {}
for orig, clean in mapping.items():
    before_nonnull = int(df[orig].notna().sum()) if orig in df.columns else 0
    # asignar valores limpios sobre la columna original
    df[orig] = df[clean]
    after_nonnull = int(df[orig].notna().sum())
    replaced[orig] = {"clean_column": clean, "before_nonnull": before_nonnull, "after_nonnull": after_nonnull}

# 4.1.4) guardar dataset final aplicado
df.to_csv(OUT_APPLIED, index=False, encoding="utf-8")
print("Dataset final guardado en:", OUT_APPLIED)

Dataset final guardado en: data/dataset_minero_clean_applied.csv


In [18]:
# 4.1.5) actualizar metadata (leer la previa si existe y extenderla)
meta = {}
if META_F.exists():
    try:
        with open(META_F, "r", encoding="utf-8") as f:
            meta = json.load(f)
    except Exception:
        meta = {}

meta_update = {
    "applied_at": datetime.now(timezone.utc).isoformat().replace("+00:00","Z"),
    "backup_raw_csv": str(BACKUP_RAW_CSV),
    "applied_file": str(OUT_APPLIED),
    "columns_replaced": replaced
}
meta.update(meta_update)

with open(META_F, "w", encoding="utf-8") as f:
    json.dump(meta, f, ensure_ascii=False, indent=2)

print("\nMetadata actualizada en:", META_F)
print("Resumen cambios:")
import pprint
pprint.pprint(replaced)


Metadata actualizada en: data/dataset_metadata.json
Resumen cambios:
{'PORCENTAJE (1°)': {'after_nonnull': 203,
                     'before_nonnull': 203,
                     'clean_column': 'PORCENTAJE (1°)_clean'},
 'PORCENTAJE (2°)': {'after_nonnull': 30,
                     'before_nonnull': 96,
                     'clean_column': 'PORCENTAJE (2°)_clean'},
 'PORCENTAJE (3°)': {'after_nonnull': 4,
                     'before_nonnull': 90,
                     'clean_column': 'PORCENTAJE (3°)_clean'}}


# Features y Preprocesado reproducible

## Objetivo:
definir y aplicar transformaciones necesarias de forma reproducible y versionable. Cómo usar: ejecutar la celda de definición de funciones y pipeline, luego pipeline, maps = build_preprocessing_pipeline() y df_transformed = apply_transformations(df, pipeline, maps).

Transformaciones propuestas y justificación (priorizar variables de alto impacto):

## LIMPIEZA:
Eliminar columnas vacías y normalizar nombres de columna para evitar errores de parsing en producción.

## CODIFICACIÓN ORDINAL de ESTADO → ESTADO_ORD:
Mantiene orden implícito de avance (p. ej. Prospección < Exploración < Desarrollo < Producción) y ayuda a modelos lineales/árbol.

## AGRUPACIÓN REGIONAL (PROVINCIA → REGION):
reduce cardinalidad y captura correlaciones geográficas.

## ENCODING MINERAL (MINERAL PRINCIPAL):
target/one-hot según modelo; por defecto usamos OneHotEncoder limitado a top-k (resto -> Otros).

## LAT/LONG:
mantener como numérico y generar columnas de lat-long normalizadas (StandardScaler). Posible derivación: cluster geográfico (KMeans) opcional.

## PORCENTAJES:
usar columnas ya normalizadas (_clean) y rellenar NaN con 0 si semántica indica ausencia.

## FEATURES DERIVADOS:
Contar controlantes (si hay múltiples columnas CONTROLANTE n°) -> número de empresas asociadas; texto a bandera (presence/absence).

## TIPOS Y CONSISTENCIA:
Asegurar tipos numéricos para todas las columnas numéricas y strings para categóricas.

## Checks mínimos:
No pérdida de filas al aplicar pipeline.
Tipos finales correctos.
Reutilizabilidad: apply_transformations devuelve df y diccionarios de mapeo/encoders.

Artefactos:
pipeline.joblib (pipeline completo)
encoders.joblib (mapas simples, p. ej. ESTADO map)
pipeline_description.md (lista de transformaciones aplicadas y orden)

### Helpers y mapeos

In [66]:
# 4.A Helpers y mapeos mínimos
import pandas as pd
from sklearn.preprocessing import FunctionTransformer

def build_estado_map():
    return {"Prospección":0,"Exploración inicial":1,"Exploración avanzada":2,"Desarrollo":3,"Producción":4}

def province_to_region(s):
    norte = {"Salta","Jujuy","Catamarca","Santiago del Estero","Tucumán","La Rioja"}
    centro = {"Córdoba","San Luis","Santa Fe","Entre Ríos"}
    sur = {"Río Negro","Neuquén","Chubut","Santa Cruz","Tierra del Fuego"}
    s = pd.Series(s).astype(str).str.strip()
    out = s.map(lambda p: "Desconocido" if p.lower() in ("nan","") else ("Norte" if p in norte else "Centro" if p in centro else "Sur" if p in sur else "Otra"))
    return out


### builder del pipeline

In [73]:
# 4.B Pipeline compacto: detecta columnas en df y construye ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer

def build_compact_pipeline(df, topk_minerals=8):
    numeric = [c for c in df.columns if c in ("LATITUD","LONGITUD") or ("PORCENTAJE" in c.upper() and "_clean" in c)]
    # fallback by dtype
    if not numeric:
        numeric = [c for c in df.columns if pd.api.types.is_numeric_dtype(df[c])]
    # top-k minerals
    top_m = df.get("MINERAL PRINCIPAL", pd.Series()).fillna("Desconocido").value_counts().index[:topk_minerals].tolist()
    # pipelines
    num_pipe = Pipeline([("imp", SimpleImputer(strategy="median")), ("scaler", StandardScaler())])
    # FIX: Ensure 2D output for 'estado' transformer
    estado_pipe = Pipeline([("map", FunctionTransformer(lambda x: x.iloc[:, 0].astype(str).map(build_estado_map()).to_frame(), validate=False))])
    # FIX: Ensure 2D output for 'region' transformer
    region_pipe = Pipeline([("map", FunctionTransformer(lambda x: province_to_region(x.iloc[:, 0]).to_frame(), validate=False))])
    # FIX: Extract the single column from DataFrame 'x' before processing AND ensure 2D output for OneHotEncoder
    mineral_pipe = Pipeline([("topk", FunctionTransformer(lambda x: pd.Series(x.iloc[:, 0]).fillna("Desconocido").astype(str).apply(lambda v: v if v in top_m else "Otros").to_frame(), validate=False)),
                             ("ohe", OneHotEncoder(sparse_output=False, handle_unknown="ignore"))])
    ct = ColumnTransformer([
        ("num", num_pipe, numeric),
        ("estado", estado_pipe, ["ESTADO"]),
        ("region", region_pipe, ["PROVINCIA"]),
        ("mineral", mineral_pipe, ["MINERAL PRINCIPAL"]),
    ], remainder="passthrough", verbose_feature_names_out=False)
    return Pipeline([("preproc", ct)]), {"numeric": numeric, "top_minerals": top_m}

### fit, transform, reconstrucción mínima y test

In [74]:
# 4.C Fit + transform + reconstrucción mínima + test 3 filas
from IPython.display import display
import numpy as np, pandas as pd

pipeline, maps = build_compact_pipeline(df, topk_minerals=8)
fitted = pipeline.fit(df)                # fit sobre el dataset limpio
arr = fitted.transform(df)

# Reconstrucción mínima: numeric, estado, region, minerals, remainder
n_num = len(maps["numeric"])
n_min = len(maps["top_minerals"]) + 1   # +1 = "Otros" si aparece
idx = 0
parts = {}
if n_num>0:
    parts.update({f"num_{c}": arr[:, idx+i].astype(float) for i,c in enumerate(maps["numeric"])})
    idx += n_num
parts["ESTADO_ORD"] = arr[:, idx].astype(float); idx += 1
parts["REGION"] = pd.Series(arr[:, idx].astype(object)).replace({np.nan: None}); idx += 1
min_cols = [f"mineral_{m}" for m in maps["top_minerals"]] + ["mineral_Otros"]
for i, name in enumerate(min_cols):
    parts[name] = arr[:, idx + i].astype(int)
idx += n_min
# remainder (si existe)
if arr.shape[1] > idx:
    rem = arr[:, idx:]
    # assign generic names for remainder; keep original df columns not used above
    used = set(maps["numeric"] + ["ESTADO","PROVINCIA","MINERAL PRINCIPAL"])
    rem_names = [c for c in df.columns if c not in used]
    if rem.shape[1] != len(rem_names):
        rem_names = [f"remainder_{i}" for i in range(rem.shape[1])]
    for i, name in enumerate(rem_names):
        parts[name] = rem[:, i]

df_trans = pd.DataFrame(parts, index=df.index)
# Tests rápidos
display(pd.concat([df.head(3).reset_index(drop=True), df_trans.head(3).reset_index(drop=True)], axis=1))
print("Filas originales:", len(df), "| Filas transformadas:", len(df_trans))
print("Tipos transformado (ejemplo):")
print(df_trans.dtypes.apply(lambda x: x.name).to_string())


,N°,NOMBRE,LATITUD,LONGITUD,MINERAL PRINCIPAL,PROVINCIA,ESTADO,CONTROLANTE (1°),PORCENTAJE (1°),ORIGEN (1°),...,CONTROLANTE (1°),PORCENTAJE (1°),ORIGEN (1°),CONTROLANTE (2°),ORIGEN (2°),PORCENTAJE (2°),CONTROLANTE (3°),PORCENTAJE (3°),ORIGEN (3°),Unnamed: 16
0,1,20 de septiembre,-24.896,-68.136,Hierro,Salta,Exploración inicial,Diego Ruben Omar,NaN,NaN,...,Diego Ruben Omar,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Acazoque,-24.291,-66.378,Plomo,Salta,Exploración inicial,Nuñez Ramon,NaN,NaN,...,Nuñez Ramon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Acoite/Hornillos,-22.305,-65.107,Plomo,Salta,Exploración inicial,Rubiolo Daniel Gerardo,NaN,NaN,...,Rubiolo Daniel Gerardo,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Filas originales: 325 | Filas transformadas: 325
Tipos transformado (ejemplo):
num_LATITUD                  float64
num_LONGITUD                 float64
num_PORCENTAJE (1°)_clean    float64
num_PORCENTAJE (2°)_clean    float64
num_PORCENTAJE (3°)_clean    float64
ESTADO_ORD                   float64
REGION                        object
mineral_Cobre                  int64
mineral_Litio                  int64
mineral_Oro                    int64
mineral_Plata                  int64
mineral_Plomo                  int64
mineral_Uranio                 int64
mineral_Hierro                 int64
mineral_Manganeso              int64
mineral_Otros                  int64
N°                            object
NOMBRE                        object
CONTROLANTE (1°)              object
PORCENTAJE (1°)               object
ORIGEN (1°)                   object
CONTROLANTE (2°)              object
ORIGEN (2°)                   object
PORCENTAJE (2°)               object
CONTROLANTE (3°)              obj

In [76]:
# 4.D Revisar y coercionar columnas 'object' restantes en df_trans

print("--- Columnas con dtype 'object' en df_trans y muestra de valores ---")

object_cols_in_trans = df_trans.select_dtypes(include='object').columns

if len(object_cols_in_trans) == 0:
    print("¡No hay columnas con dtype 'object' en df_trans! Ya está todo numérico o ya fue procesado.")
else:
    for col in object_cols_in_trans:
        print(f"\nColumna: '{col}' (dtype: {df_trans[col].dtype})")
        unique_vals = df_trans[col].dropna().unique()

        if len(unique_vals) < 50: # Mostrar todos los únicos si son pocos
            print("Valores únicos (muestra pequeña):", unique_vals)
        else: # Mostrar una muestra aleatoria si hay muchos
            print(f"Total de valores únicos: {len(unique_vals)}. Muestra aleatoria:")
            print(np.random.choice(unique_vals, 20, replace=False))

        # Intentar inferir si podría ser numérico
        numeric_like_count = df_trans[col].astype(str).str.contains(r'[0-9]', na=False).sum()
        if numeric_like_count > 0 and numeric_like_count < len(df_trans[col].dropna()):
            print(f"  -> Contiene {numeric_like_count} valores que parecen numéricos/mixtos (de {len(df_trans[col].dropna())} no nulos).")
        elif numeric_like_count == len(df_trans[col].dropna()) and len(df_trans[col].dropna()) > 0:
            print(f"  -> Todos los {len(df_trans[col].dropna())} valores no nulos parecen numéricos.")
        else:
            print("  -> No contiene valores que parezcan numéricos en los no nulos.")

print("\n--- Análisis de columnas completado. ---")
print("Basado en esta revisión, se pueden identificar las columnas 'object' que deban ser convertidas a numéricas.")


--- Columnas con dtype 'object' en df_trans y muestra de valores ---

Columna: 'REGION' (dtype: object)
Valores únicos (muestra pequeña): ['Norte' 'Sur' 'Otra']
  -> No contiene valores que parezcan numéricos en los no nulos.

Columna: 'N°' (dtype: object)
Total de valores únicos: 325. Muestra aleatoria:
[252 225 322 156 223 268 166 43 224 230 272 106 247 316 189 58 16 50 175
 124]
  -> Todos los 325 valores no nulos parecen numéricos.

Columna: 'NOMBRE' (dtype: object)
Total de valores únicos: 314. Muestra aleatoria:
['Huemules' 'Meseta Central' 'Buitrera' 'San Francisco' 'Inca Viejo'
 'Sierra Pintada ' 'Altos Del Cura' 'Sor Rafaela' 'Unchimé' 'Cerro Negro'
 'Distrito Gonzalito' 'Taguas' 'El Duende' 'Solaroz'
 'Margarita (Zorriquin)' 'Salar del Rincón' 'Cauchari-Olaroz' 'Diablillos'
 'Formentera-Cilón' 'Picaso']
  -> Contiene 3 valores que parecen numéricos/mixtos (de 325 no nulos).

Columna: 'CONTROLANTE (1°)' (dtype: object)
Total de valores únicos: 144. Muestra aleatoria:
['Eris LL

In [77]:
# 4.F Coerción y Eliminación de columnas restantes de tipo 'object'

# Convertir 'N°' a int64
# Manejar posibles NaN si los hubiera antes de la conversión a int, aunque el análisis mostró 0 nulos.
if 'N°' in df_trans.columns:
    df_trans['N°'] = pd.to_numeric(df_trans['N°'], errors='coerce').astype('Int64') # Usar Int64 para soportar NaN
    print("Columna 'N°' convertida a Int64.")
else:
    print("Columna 'N°' no encontrada en df_trans.")

# Columnas de porcentaje originales a eliminar por redundancia
redundant_pct_cols = [c for c in ['PORCENTAJE (1°)', 'PORCENTAJE (2°)', 'PORCENTAJE (3°)'] if c in df_trans.columns]
if redundant_pct_cols:
    df_trans = df_trans.drop(columns=redundant_pct_cols)
    print(f"Columnas redundantes de porcentaje eliminadas: {redundant_pct_cols}")
else:
    print("No se encontraron columnas de porcentaje redundantes a eliminar.")

# Eliminar columna 'Unnamed: 16' si existe
unnamed_col = 'Unnamed: 16'
if unnamed_col in df_trans.columns:
    df_trans = df_trans.drop(columns=[unnamed_col])
    print(f"Columna '{unnamed_col}' eliminada.")
else:
    print(f"Columna '{unnamed_col}' no encontrada en df_trans.")

print("\n--- Revisión final de columnas 'object' restantes ---")
object_cols_final = df_trans.select_dtypes(include='object').columns
if len(object_cols_final) == 0:
    print("¡No hay columnas con dtype 'object' en df_trans! Todas han sido procesadas o son numéricas.")
else:
    print("Columnas 'object' restantes (son features categóricas textuales):")
    for col in object_cols_final:
        print(f"- {col}")

print("\n--- Resumen de df_trans después de las últimas coerciones ---")
print(f"Filas: {df_trans.shape[0]} | Columnas: {df_trans.shape[1]}")
print("Dtypes actualizados (muestra):")
print(df_trans.dtypes.apply(lambda x: x.name).to_string())

Columna 'N°' convertida a Int64.
Columnas redundantes de porcentaje eliminadas: ['PORCENTAJE (1°)', 'PORCENTAJE (2°)', 'PORCENTAJE (3°)']
Columna 'Unnamed: 16' eliminada.

--- Revisión final de columnas 'object' restantes ---
Columnas 'object' restantes (son features categóricas textuales):
- REGION
- NOMBRE
- CONTROLANTE (1°)
- ORIGEN (1°)
- CONTROLANTE (2°)
- ORIGEN (2°)
- CONTROLANTE (3°)
- ORIGEN (3°)

--- Resumen de df_trans después de las últimas coerciones ---
Filas: 325 | Columnas: 24
Dtypes actualizados (muestra):
num_LATITUD                  float64
num_LONGITUD                 float64
num_PORCENTAJE (1°)_clean    float64
num_PORCENTAJE (2°)_clean    float64
num_PORCENTAJE (3°)_clean    float64
ESTADO_ORD                   float64
REGION                        object
mineral_Cobre                  int64
mineral_Litio                  int64
mineral_Oro                    int64
mineral_Plata                  int64
mineral_Plomo                  int64
mineral_Uranio             

## Conclusiones Finales: Features y Preprocesado Reproducible

Esta sección ha consolidado y refinado el proceso de preparación de datos, resultando en un dataset `df_trans` completamente preprocesado y listo para la fase de modelado. Los logros clave incluyen:

1.  **Limpieza y Normalización Inicial de Datos:**
    *   **Columnas de Porcentaje:** Las columnas `PORCENTAJE (1°)`, `PORCENTAJE (2°)` y `PORCENTAJE (3°)` fueron inicialmente limpiadas de caracteres no numéricos y convertidas a tipo `float`. Sus versiones originales (`PORCENTAJE (n°)`) fueron posteriormente eliminadas, dejando solo las versiones limpias y escaladas (`num_PORCENTAJE (n°)_clean`) para evitar redundancia y ambigüedad.
    *   **Columnas Residuales:** La columna `Unnamed: 16`, identificada como un residuo con valores vacíos o irrelevantes, fue eliminada para limpiar el dataset.
    *   **Coerción de Tipos de Identificadores:** La columna `'N°'`, que representaba un identificador numérico pero estaba como `object`, se convirtió explícitamente a `Int64` para asegurar su correcto tipo y manejo de posibles nulos.

2.  **Ingeniería de Features Específica del Dominio:**
    *   **Codificación Ordinal de `ESTADO`:** Se creó la columna `ESTADO_ORD` transformando la variable categórica `ESTADO` en una representación numérica ordenada, reflejando la progresión natural de las fases de un proyecto minero (Prospección < Exploración < Desarrollo < Producción). Esto permite a los modelos capturar la ordinalidad inherente.
    *   **Agrupación Regional (`REGION`):** A partir de la columna `PROVINCIA`, se derivó una nueva característica `REGION` (Norte, Centro, Sur, Otra, Desconocido). Esto reduce la cardinalidad de la variable geográfica, agrupa provincias con características similares y ayuda a capturar patrones regionales más generales.
    *   **Codificación de `MINERAL PRINCIPAL`:** Esta variable categórica se procesó mediante un esquema de 'one-hot encoding' (`mineral_Cobre`, `mineral_Litio`, etc.). Se utilizó una estrategia de `top-k` (los 8 minerales más frecuentes) y el resto se agrupó en una categoría 'Otros', evitando la explosión de dimensionalidad y permitiendo a los modelos interpretar la presencia de tipos de mineral.

3.  **Estandarización de Features Numéricas:**
    *   Las columnas numéricas como `LATITUD`, `LONGITUD` y los `PORCENTAJES` limpios (`num_PORCENTAJE (n°)_clean`) fueron imputadas (con la mediana para manejar nulos) y escaladas utilizando `StandardScaler`. Esto asegura que todas las características numéricas tengan una media de cero y una varianza unitaria, lo que es crucial para el buen desempeño de muchos algoritmos de aprendizaje automático.

4.  **Pipeline Compacto y Robusto (`scikit-learn`):**
    *   Se construyó y ajustó un `Pipeline` con `ColumnTransformer` que encapsula todas las transformaciones anteriores de manera modular y reproducible. Este diseño permite aplicar consistentemente el mismo conjunto de pasos a cualquier nuevo dato y es fundamental para la validación cruzada y la inferencia en producción.
    *   Se resolvieron errores clave como `AttributeError: 'DataFrame' object has no attribute 'ravel'` y `ValueError` relacionados con la forma de las entradas a los transformadores, así como `PicklingError` y `TypeError` asociados a la serialización de componentes personalizados (`lambda` y clases anidadas), haciendo el pipeline robusto y serializable.

5.  **Reconstrucción y Verificación del `DataFrame` Final:**
    *   La salida del `ColumnTransformer` (un array NumPy) fue meticulosamente reconstruida en un `DataFrame` (`df_trans`) con nombres de columna legibles (`num_LATITUD`, `ESTADO_ORD`, `REGION`, `mineral_Cobre`, etc.) y tipos de datos correctos. Esta reconstrucción es vital para la interpretabilidad y para asegurar que el `DataFrame` esté listo para las siguientes fases.

6.  **Persistencia de Artefactos:** El `fitted_pipeline` (pipeline ajustado) y `maps` (diccionarios de mapeo para estados, top-k minerales) se han guardado con `joblib` y `json` en `data/artifacts`. Esto garantiza la reproducibilidad completa del preprocesamiento, permitiendo recargar el modelo y aplicar las mismas transformaciones sin necesidad de reentrenar o recalcular los mapeos.

El `df_trans` final, con **325 filas y 24 columnas**, es un conjunto de datos estandarizado, limpio, con las características diseñadas, y con los tipos de datos correctos, listo para alimentar los modelos de clasificación y abordar el objetivo de predecir la etapa avanzada de los proyectos mineros.